In [1]:
import time_evolution as te
import help_functions as hf
import quasi_stationary_investigation as qsi
import cyclic_blockade as cb

import numpy as np
import setup as set
from copy import copy

import cyclic_blockade as cb

import os

def data_dir(t_set, data_format=0):
    format_dir = "format_{}".format(data_format)
    
    # Define the base directory where the data will be saved
    base_dir = "/Users/ma0274ni/Documents/projects/majorana_box/data/fusion_rule"
    cwd = os.getcwd()

    # Define the parameters that the data depends on
    params = param_dict()

    angles = t_set.theta_arr()
    for i, key in enumerate(['theta0', 'theta1', 'theta2', 'theta3']):
        params[key] = angles[i]

    overlaps = t_set.overlap_arr()
    for i, key in enumerate(['eps01', 'eps12', 'eps23', 'eps_abs_0', 'eps_abs_1', 'eps_abs_2', 'eps_abs_3']):
        params[key] = overlaps[i]

    # Create a directory with a specific name based on the parameters
    dir_path = os.path.join(base_dir, format_dir)
    os.makedirs(dir_path, exist_ok=True)

    # Save the data to a file with a specific name based on the parameters
    data = [1, 2, 3, 4, 5]
    file_name = "data_{}.npy".format("_".join(["{}-{}".format(k, v) for k, v in params.items()]))
    file_path = os.path.join(dir_path, file_name)
    return file_path

def save_data(data, t_set, data_format=0):
    file_path = data_dir(t_set, data_format)
    np.save(file)
    print('Data saved in: ', file_path)

def load_data(t_set, data_format=0):
    file_path = data_dir(t_set, data_format)
    try:
        data = np.load(file_path)
        print('Data loaded from: ', file_path)
    except FileNotFoundError:
        print(f'File {filepath} not found')

def param_dict():
    params = {
        "theta0": 0.0,
        "theta1": 0.0,
        "theta2": 0.0,
        "theta3": 0.0,
        "eps01": 0.0,
        "eps12": 0.0,
        "eps23": 0.0,
        "eps_abs_0": 0.0,
        "eps_abs_1": 0.0,
        "eps_abs_2": 0.0,
        "eps_abs_3": 0.0,
        "routine": "1vN"
    }
    return params

In [2]:
t_set = set.create_transport_setup()
file_path = data_dir(t_set)
print(file_path)

/Users/ma0274ni/Documents/projects/majorana_box/data/fusion_rule/format_0/data_theta0-0.9424777960769379_theta1-0.0_theta2-0.0_theta3-0.0_eps01-0.001_eps12-0.0015_eps23-0.002_eps_abs_0-0.0005_eps_abs_1-0.001_eps_abs_2-0.0015_eps_abs_3-0.002_routine-1vN.npy


In [3]:
waiting_time = 1e2
n = 1
pre_run = 1e3
qs_desc = False

initialization = 2   
t_set_z = set.create_transport_setup()
timescale = 1e3
waiting_times = np.linspace(1, 4, 5)*timescale
waiting_times = np.power(10, np.linspace(-1, 6.5, 40) )
include_Irem = True

charge_random = []
charge_maj = []
angle_arr = []
for idx in range(11):
    angles = np.random.rand(4)*np.pi*2
    if idx==0:
        t_set_z.model = 1
    else:
        t_set_z.model = 2
        angle_arr.append(angles)
        print(angles)
    
    t_set_z.th0, t_set_z.th1, t_set_z.th2, t_set_z.th3 = angles
    t_set_z.initialize_leads()
    t_set_z.adjust_to_z_blockade()
    print(t_set_z.gamma_00, t_set_z.gamma_01, t_set_z.gamma_02)
    t_set_z.initialize_box()
    t_set_z.connect_box()

    t_set_x = t_set_z.copy()

    t_set_x.adjust_to_x_blockade()
    print(t_set_x.gamma_00, t_set_x.gamma_01, t_set_x.gamma_02)
    t_set_x.initialize_leads()
    t_set_x.initialize_box()
    t_set_x.connect_box()

    sys_z = t_set_z.build_qmeq_sys()
    sys_x = t_set_x.build_qmeq_sys()

    sys_z.solve(qdq=False, rotateq=False)
    sys_x.solve(qdq=False, rotateq=False)
    print(sys_z.current, sys_x.current )

    rho0, sys_z, sys_x, current_fct_z, current_fct_x, time_evo_rho_z, time_evo_rho_x = cb.initialize_cycle_fcts(t_set_z, t_set_x, qs_desc, initialization, lead=0)
    res = np.array([cb.charge_transmission_cycle(current_fct_z, current_fct_x, time_evo_rho_z, time_evo_rho_x, rho0, n, waiting_time, sys_z, pre_run=pre_run, include_Irem=include_Irem) for waiting_time in waiting_times ] )

    if idx == 0:
        charge_maj.append(res)
    else:
        charge_random.append(res)

charge_maj = np.array(charge_maj)
charge_random = np.array(charge_random)

1.0 1.0 0.0
0.0 1.0 1.0
[ 1.77934189e-07 -1.77934189e-07] [ 7.11274903e-07 -7.11274903e-07]
Pre-running system 1000.0 cycles
Cycle 1 of 1
Pre-running system 1000.0 cycles
Cycle 1 of 1
Pre-running system 1000.0 cycles
Cycle 1 of 1
Pre-running system 1000.0 cycles
Cycle 1 of 1
Pre-running system 1000.0 cycles
Cycle 1 of 1
Pre-running system 1000.0 cycles
Cycle 1 of 1
Pre-running system 1000.0 cycles
Cycle 1 of 1
Pre-running system 1000.0 cycles
Cycle 1 of 1
Pre-running system 1000.0 cycles
Cycle 1 of 1
Pre-running system 1000.0 cycles
Cycle 1 of 1
Pre-running system 1000.0 cycles
Cycle 1 of 1
Pre-running system 1000.0 cycles
Cycle 1 of 1
Pre-running system 1000.0 cycles
Cycle 1 of 1
Pre-running system 1000.0 cycles
Cycle 1 of 1
Pre-running system 1000.0 cycles
Cycle 1 of 1
Pre-running system 1000.0 cycles
Cycle 1 of 1
Pre-running system 1000.0 cycles
Cycle 1 of 1
Pre-running system 1000.0 cycles
Cycle 1 of 1
Pre-running system 1000.0 cycles
Cycle 1 of 1
Pre-running system 1000.0 cycles
C

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams.update({"text.usetex": True, "font.family": "serif"})

fig, ax = plt.subplots(1,1, figsize=(6,4))
timescale = (t_set_z.eps_abs_0)**(-1)
timescale = 1

data = charge_random
#data = charge
ax.plot(waiting_times/timescale, np.transpose(data), c='gray' )
ax.plot(waiting_times, np.transpose(charge_maj), c='k')

fs = 16

#ax.legend([r'$\theta_2 = {:.2f} \, \pi$'.format(angle/np.pi) for angle in angles])
ax.set_xlabel(r'$\tau \, [1/\Gamma]$', size=fs)
ax.set_ylabel('Transmitted charge [e]', size=fs)
ax.set_ylim([0,1.5])
ax.set_xlim([0.1, 1.5e6])
ax.set_xscale('log')
ax.set_xticks(np.power(10, range(0, 7, 2) ) )
ax.set_yticks(np.linspace(0, 1.5, 4) )
ax.tick_params(labelsize=fs)
# ax.grid()
plt.tight_layout()

#Save created plot as regular picture
#plt.savefig('/Users/ma0274ni/Documents/projects/majorana_box/papers/fusion/plots/ABS-box_charge-blocktime_f1=1-f2=1-f3=1_th0=0-th1=0-th2=sweep_1vN.pdf', format='pdf')
#Save as zoom
#plt.savefig('/Users/ma0274ni/Documents/projects/majorana_box/papers/fusion/plots/ABS-box_charge-blocktime_f1=1-f2=1-f3=1_th0=0-th1=0-th2=sweep_1vN_zoom.pdf', format='pdf')
#Save as random sampling
plt.savefig('/Users/ma0274ni/Documents/projects/majorana_box/papers/fusion/plots/ABS-box_charge-blocktime_f1=1-f2=1-f3=1_thetas-randomized_1vN.pdf', format='pdf')
#Save as th3-sweep
#plt.savefig('/Users/ma0274ni/Documents/projects/majorana_box/papers/fusion/plots/ABS-box_charge-blocktime_f1=1-f2=1-f3=1_th3=sweep_1vN.pdf', format='pdf')

In [5]:
print(file_path)

/Users/ma0274ni/Documents/projects/majorana_box/data/fusion_rule/format1/data_theta0-0.0_theta1-0.0_theta2-1.413716694115407_theta3-0.0_eps01-0.002_eps12-0.001_eps23-0.003_eps_abs_0-0.001_eps_abs_1-0.002001_eps_abs_2-0.003_eps_abs_3-0.004_routine-1vN.npy


In [3]:
waiting_time = 1e2
n = 1
pre_run = 1e3
qs_desc = False

initialization = 2              ### Initialization routines 0: stationary state of system;
                                ### 1: 1 at first occupation, 0 otherwise; 2: maximally mixed state

t_set_z = set.create_transport_setup()

timescale = (t_set_z.eps_abs_0)**(-1)
timescale = 1e3

waiting_times = np.linspace(1, 4, 4)*timescale
waiting_times = np.power(10, np.linspace(-1, 3, 10) )

include_Irem = True

angle_arr  = [[0, 0, 0, 0.99*np.pi/2]]
angle_arr  = np.array([ [0, 0, 0, fac*np.pi] for fac in np.linspace(0, 1, 10) ])
charge  = []
for angles in angle_arr:
    print(angles/np.pi)
    t_set_z.th0, t_set_z.th1, t_set_z.th2, t_set_z.th3 = angles
    t_set_z.initialize_leads()
    t_set_z.adjust_to_z_blockade()
    t_set_z.initialize_box()
    t_set_z.connect_box()

    t_set_x = t_set_z.copy()

    t_set_x.adjust_to_x_blockade()
    t_set_x.initialize_leads()
    t_set_x.initialize_box()
    t_set_x.connect_box()

    sys_z = t_set_z.build_qmeq_sys()
    sys_x = t_set_x.build_qmeq_sys()

    sys_z.solve(qdq=False, rotateq=False)
    sys_x.solve(qdq=False, rotateq=False)
    print(sys_z.current, sys_x.current )


    qsi.analytical_prediction(t_set_z.th2 )

    rho0, sys_z, sys_x, current_fct_z, current_fct_x, time_evo_rho_z, time_evo_rho_x = cb.initialize_cycle_fcts(t_set_z, t_set_x, qs_desc, initialization, lead=0)
    
    charge.append(np.array([cb.charge_transmission_cycle(current_fct_z, current_fct_x, time_evo_rho_z, time_evo_rho_x, rho0, n, waiting_time, sys_z, pre_run=pre_run, include_Irem=include_Irem) for waiting_time in waiting_times ] ) )

charge = np.array(charge)

[0. 0. 0. 0.]
[ 1.77934189e-07 -1.77934189e-07] [ 7.11274903e-07 -7.11274903e-07]
ePlus squared:  1.0
eMinus squared:  1.0
Analytically predicted charge:  0.5
Pre-running system 1000.0 cycles
Cycle 1 of 1
Pre-running system 1000.0 cycles
Cycle 1 of 1
Pre-running system 1000.0 cycles
Cycle 1 of 1
Pre-running system 1000.0 cycles
Cycle 1 of 1
Pre-running system 1000.0 cycles
Cycle 1 of 1
Pre-running system 1000.0 cycles
Cycle 1 of 1
Pre-running system 1000.0 cycles
Cycle 1 of 1
Pre-running system 1000.0 cycles
Cycle 1 of 1
Pre-running system 1000.0 cycles
Cycle 1 of 1
Pre-running system 1000.0 cycles
Cycle 1 of 1
[0.         0.         0.         0.11111111]
[ 1.77934189e-07 -1.77934189e-07] [ 7.11274903e-07 -7.11274903e-07]
ePlus squared:  1.0
eMinus squared:  1.0
Analytically predicted charge:  0.5
Pre-running system 1000.0 cycles
Cycle 1 of 1
Pre-running system 1000.0 cycles
Cycle 1 of 1
Pre-running system 1000.0 cycles
Cycle 1 of 1
Pre-running system 1000.0 cycles
Cycle 1 of 1
Pre-ru